In [2]:
from elasticsearch import Elasticsearch, helpers, exceptions
from getpass import getpass

#Connect to the elastic cloud server
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

/Users/iulia/python/cheat sheet/elasticsearch-python/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [43]:
model_id = ".elser_model_2"

# delete model if already downloaded and deployed
try:
    client.ml.delete_trained_model(model_id=model_id, force=True)
    print("Model deleted successfully, We will proceed with creating one")
except exceptions.NotFoundError:
    print("Model doesn't exist, but We will proceed with creating one")

# Creates the ELSER model configuration. Automatically downloads the model if it doesn't exist.
client.ml.put_trained_model(
    model_id=".elser_model_2", input={"field_names": ["text_field"]}
)

Model deleted successfully, We will proceed with creating one


/var/folders/k3/sm4kh0y91fs5_fft4375wwh00000gn/T/ipykernel_51317/16398611.py:5: ElasticsearchWarning: The default [remove_binary] value of 'false' is deprecated and will be set to 'true' in a future release. Set [remove_binary] explicitly to 'true' or 'false' to ensure no behavior change.
  client.ml.delete_trained_model(model_id=model_id, force=True)


ObjectApiResponse({'model_id': '.elser_model_2', 'model_type': 'pytorch', 'model_package': {'packaged_model_id': 'elser_model_2', 'model_repository': 'https://ml-models.elastic.co', 'minimum_version': '11.0.0', 'size': 438123914, 'sha256': '2e0450a1c598221a919917cbb05d8672aed6c613c028008fedcd696462c81af0', 'metadata': {}, 'tags': [], 'vocabulary_file': 'elser_model_2.vocab.json'}, 'created_by': 'api_user', 'version': '12.0.0', 'create_time': 1711453868319, 'model_size_bytes': 0, 'estimated_operations': 0, 'license_level': 'platinum', 'description': 'Elastic Learned Sparse EncodeR v2', 'tags': ['elastic'], 'metadata': {}, 'input': {'field_names': ['text_field']}, 'inference_config': {'text_expansion': {'vocabulary': {'index': '.ml-inference-native-000002'}, 'tokenization': {'bert': {'do_lower_case': True, 'with_special_tokens': True, 'max_sequence_length': 512, 'truncate': 'first', 'span': -1}}}}, 'location': {'index': {'name': '.ml-inference-native-000002'}}})

In [7]:
# Is model downloaded and ready to deploy?

import time

while True:
    status = client.ml.get_trained_models(
        model_id=".elser_model_2", include="definition_status"
    )

    if status["trained_model_configs"][0]["fully_defined"]:
        print("ELSER Model is downloaded and ready to be deployed.")
        break
    else:
        print("ELSER Model is downloaded but not ready to be deployed.")
    time.sleep(5)

ELSER Model is downloaded and ready to be deployed.


In [50]:
client.ml.start_trained_model_deployment(
    model_id=".elser_model_2", 
    deployment_id="elser_inference_1",
    number_of_allocations=1, 
    wait_for="starting"
)

# These will be set to default: 
# 'threads_per_allocation': 1, 
# 'number_of_allocations': 1, 
# 'queue_capacity': 1024

ObjectApiResponse({'assignment': {'task_parameters': {'model_id': '.elser_model_2', 'deployment_id': 'elser_inference_1', 'model_bytes': 438123914, 'threads_per_allocation': 1, 'number_of_allocations': 1, 'queue_capacity': 1024, 'cache_size': '438123914b', 'priority': 'normal', 'per_deployment_memory_bytes': 0, 'per_allocation_memory_bytes': 0}, 'routing_table': {'WmcykdAZQuuVWFZJM4162A': {'current_allocations': 1, 'target_allocations': 1, 'routing_state': 'starting', 'reason': ''}}, 'assignment_state': 'starting', 'start_time': '2024-03-26T13:38:30.915944514Z', 'max_assigned_allocations': 1}})

In [6]:
client.ml.start_trained_model_deployment(
    model_id=".elser_model_2", 
    deployment_id="elser_inference_configured", 
    number_of_allocations=3,
    threads_per_allocation=8, 
    queue_capacity=7000, 
    timeout='1m', 
    wait_for="starting",
    )

ObjectApiResponse({'assignment': {'task_parameters': {'model_id': '.elser_model_2', 'deployment_id': 'elser_inference_configured', 'model_bytes': 438123914, 'threads_per_allocation': 8, 'number_of_allocations': 3, 'queue_capacity': 7000, 'cache_size': '438123914b', 'priority': 'normal', 'per_deployment_memory_bytes': 0, 'per_allocation_memory_bytes': 0}, 'routing_table': {}, 'assignment_state': 'starting', 'reason': 'Could not assign (more) allocations on node [WmcykdAZQuuVWFZJM4162A]. Reason: This node has insufficient allocated processors. Available processors [8], free processors [3], processors required for each allocation of this model [8]|Could not assign (more) allocations on node [lLiH3Mj2ThOJCriaSx0deg]. Reason: This node has insufficient allocated processors. Available processors [8], free processors [0], processors required for each allocation of this model [8]', 'start_time': '2024-04-03T11:41:09.625226789Z', 'max_assigned_allocations': 0}})

In [82]:
response = client.ml.infer_trained_model(
    model_id="elser_inference_configured",
    #deployment_id="elser_inference_1",
    body={
        "docs": [
            {
                "text_field": "This movie was totally awesome and had loads of adventure"
            }
        ]
    }
)
print(response["inference_results"][0]["predicted_value"])

NotFoundError: NotFoundError(404, 'resource_not_found_exception', 'Could not find trained model [elser_inference_configured]')

In [26]:
# Is model deployed successfuly and ready for inference?

while True:
    status = client.ml.get_trained_models_stats(
        model_id=".elser_model_2",
    )
    if status["trained_model_stats"][0]["deployment_stats"]["state"] == "started":
        print("ELSER Model has been successfully deployed.")
        break
    else:
        print("ELSER Model is currently being deployed.")
    time.sleep(5)

ELSER Model has been successfully deployed.


In [27]:
response = client.ml.infer_trained_model(
    model_id=".elser_model_2",
    body={
        "docs": [
            {
                "text_field": "This movie was totally awesome and had loads of adventure"
            }
        ]
    }
)
print(response["inference_results"][0]["predicted_value"])

{'awesome': 1.8774871, 'adventure': 1.8254844, 'movie': 1.7159772, 'amazing': 1.5583062, 'totally': 1.349275, 'film': 0.96663076, 'this': 0.8041103, 'stunt': 0.73504764, 'scene': 0.7298112, 'gross': 0.7292234, 'completely': 0.7290972, 'picture': 0.7272054, 'cartoon': 0.70581627, 'thriller': 0.68924236, 'incredible': 0.68247473, 'lots': 0.6567157, 'plenty': 0.6550607, 'sequel': 0.6424385, 'had': 0.6287307, 'kid': 0.62860245, 'disney': 0.6090568, 'much': 0.6006187, 'adventures': 0.5567534, 'zombie': 0.5415081, 'movies': 0.5409077, 'monster': 0.53614014, 'epic': 0.5066528, 'was': 0.50616497, 'novel': 0.504593, 'full': 0.4988839, 'loads': 0.49524742, 'absolutely': 0.4794867, 'great': 0.47640574, 'enough': 0.46359646, 'bomb': 0.42215556, 'alien': 0.41104022, 'fish': 0.40768614, 'horror': 0.40727267, 'story': 0.39797214, 'oscar': 0.3953349, 'anime': 0.3423973, 'ralph': 0.3414993, 'exclusive': 0.3293829, 'soundtrack': 0.3231836, 'explore': 0.31846526, 'ocean': 0.29446197, 'and': 0.27902943, '

In [28]:
# Elser embeddings pipeline & reindex
client.ingest.put_pipeline(
    id="elser-2-ingest-pipeline-optimized",
    description="Ingest pipeline for ELSER",
    processors=[
        {
            "inference": {
                "model_id": ".elser_model_2",
                "input_output": [
                    {"input_field": "plot", "output_field": "plot_embedding"}
                ],
            }
        }
    ],
)

client.indices.delete(index="elser-movies", ignore_unavailable=True)
client.indices.create(
    index="elser-movies",
    mappings={
        "properties": {
            "plot": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "plot_embedding": {"type": "sparse_vector"},
        }
    },
)
client.reindex(
    source={"index": "movies"},
    dest={"index": "elser-movies", "pipeline": "elser-2-ingest-pipeline-optimized"},
    wait_for_completion=False
)


ObjectApiResponse({'task': 'eclQBhHoS0CN09g-_bZM5w:28525285'})

In [30]:
# Query
response = client.search(
    index="elser-movies",
    size=3,
    query={
        "text_expansion": {
            "plot_embedding": {
                "model_id": model_id,
                "model_text": "investigation",
            }
        }
    },
)

for hit in response["hits"]["hits"]:
    doc_id = hit["_id"]
    score = hit["_score"]
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Score: {score}\nTitle: {title}\nPlot: {plot}\n")

Score: 6.4037366
Title: se7en
Plot: Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.

Score: 3.6703353
Title: the departed
Plot: An undercover cop and a mole in the police attempt to identify each other while infiltrating an Irish gang in South Boston.

Score: 2.9359112
Title: the usual suspects
Plot: A sole survivor tells of the twisty events leading up to a horrific gun battle on a boat, which began when five criminals met at a seemingly random police lineup.



In [65]:
response = client.search(index = "hp_books", size=5000)
docs = []
for line in response["hits"]["hits"]:
    docs.append({"text_field" : line["_source"]["text_field"]})

len(docs)

5000

In [67]:
docs[0]

{'text_field': 'THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much '}

In [ ]:
!docker pull docker.elastic.co/eland/eland:8.11.1


In [17]:
response = client.search(index = "elser-movies", size=5000)

docs = []
for line in response["hits"]["hits"]:
    docs.append({"text_field" : line["_source"]["plot_embedding"]})

response = client.ml.infer_trained_model(model_id=model_id, docs=docs, timeout='5m')

NameError: name 'model_id' is not defined

In [74]:
print(response)

{'took': 9, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'hp_books', '_id': '-emUX4wBBBHWSNqPCvxS', '_score': 1.0, '_source': {'text_field': 'THE BOY WHO LIVED Mr and Mrs Dursley of number four Privet Drive were proud to say that they were perfectly normal thank you very much '}}, {'_index': 'hp_books', '_id': '-umUX4wBBBHWSNqPCvxS', '_score': 1.0, '_source': {'text_field': 'They were the last people youd expect to be involved in anything strange or mysterious because they just didnt hold with such nonsense '}}, {'_index': 'hp_books', '_id': '--mUX4wBBBHWSNqPCvxS', '_score': 1.0, '_source': {'text_field': 'Mr Dursley was the director of a firm called Grunnings which made drills '}}, {'_index': 'hp_books', '_id': '_OmUX4wBBBHWSNqPCvxS', '_score': 1.0, '_source': {'text_field': 'He was a big beefy man with hardly any neck although he did have a very lar

In [32]:
# Elser embeddings pipeline & reindex
client.ingest.put_pipeline(
    id="elser-2-ingest-pipeline-optimized",
    description="Ingest pipeline for ELSER with a lot more requests",
    processors=[
        {
            "inference": {
                "model_id": model_id,
                "input_output": [
                    {"input_field": "text_field", "output_field": "text_embedding"}
                ],
            }
        }
    ],
)

#client.indices.delete(index="hp-elser-optimized")
client.indices.create(
    index="hp-elser-optimized",
    mappings={
        "properties": {
            "text_field": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
            "text_embedding": {"type": "sparse_vector"},
        }
    },
)
client.reindex(
    source={"index": "hp_books"},
    dest={"index": "hp-elser-optimized", "pipeline": "elser-2-ingest-pipeline-optimized"},
    wait_for_completion=False,
)


ObjectApiResponse({'task': 'eclQBhHoS0CN09g-_bZM5w:28532199'})


"elser-2-ingest-pipeline": 
"count": 12,
"time_in_millis": 9259, --> just under 10s for the 12 document pipeline


GET /_ingest/pipeline/my-pipeline-id




GET _nodes/hot_threads 
100.0% [cpu=3.5%, other=96.5%]
ml.allocated_processors=16

## Bottleneck Opportunities Phase 1 - downloading the model 

### ML node size

Ensure the ML node configuration is appropriate for the model you want to use. 

https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html 
> The minimum dedicated ML node size for deploying and using the ELSER model is 4 GB in Elasticsearch Service if deployment autoscaling is turned off. 
>
> The minimum dedicated ML node size for deploying and using the natural language processing models is 16 GB in Elasticsearch Service if deployment autoscaling is turned off. 
>
>Turning on autoscaling is recommended because it allows your deployment to dynamically adjust resources based on demand. Better performance can be achieved by using more allocations or more threads per allocation, which requires bigger ML nodes. Autoscaling provides bigger nodes when required. If autoscaling is turned off, you must provide suitably sized nodes yourself.


When the ML node size is not large enough (at least 4GB for ELSER, or at least 16GM for natural language processing models) when you try to deploy the mdoel you will get this error message:

```ApiError(429, 'status_exception', 'Could not start deployment because no ML nodes with sufficient capacity were found')```

You can check the status of your model in the UI:
![error](/img/trained%20models%20UI.png)

Or by running 

### Threads & Allocations

https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-deploy-model.html
>It is recommended to fine-tune each deployment based on its specific purpose. To improve ingest performance, increase throughput by adding more allocations to the deployment. 
>It increases the number of inference requests that can be performed in parallel. 
>For improved search speed, increase the number of threads per allocation.
>Increasing the number of threads generally increases the speed of inference requests. The value of this setting must not exceed the number of available allocated processors per node.

>threads_per_allocation

>(Optional, integer) Sets the number of threads used by each model allocation during inference. This generally increases the speed per inference request. The inference process is a compute-bound process; threads_per_allocations must not exceed the number of available allocated processors per node. Defaults to 1. Must be a power of 2. Max allowed value is 32.

See benchmarking information on how performance increases for ELSER with number of allocations: https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-elser.html#_elser_v2_2 


# Important!
You can choose threads and allocations for your model at deployment time, however these cannot be bigger than the threads & allocations avaialble on your ML node. Therefore it is important to set the appropriate configurations before attempting to download and deploy the model.





## Bottleneck Opportunities Phase 2: Deploying the model

### Wait for... checking for completion before the next step

```
client.ml.start_trained_model_deployment(
    model_id=".elser_model_2", 
    deployment_id="elser_inference_1",
    number_of_allocations=1, 
    wait_for="starting"
)
```

In this case `wait_for` defaults to `started`, which means you will get a response when the model has finished downloading and is successfully deployed. You can change this to:

Another thing to mind at this stage, is that running commands before the previous step has finished running will result in erros like the following:

This can happen when you try to deploy a model that hasn't been fully downloaded yet:
Error:
> "Model definition truncated. Unable to deserialize trained model definition [.elser_model_2]"

Solution:
You can check the status of the model in the UI or via the following command to ensure the model is ready to be used:
```
status = client.ml.get_trained_models(
    model_id=".elser_model_2", include="definition_status"
    )
```

You can deploy the model when `status["trained_model_configs"][0]["fully_defined"] == True`

Or when you try to run inference on a model that hasn't been fully deployed yet:
Error:
> "404, 'resource_not_found_exception', 'Could not find trained model [elser_inference_configured]'"

Solution:
```
status = client.ml.get_trained_models_stats(
    model_id=".elser_model_2",
    )
```
You can start making calls to the model when `status["trained_model_stats"][0]["deployment_stats"]["state"] == "started"` 




### Timeout
Model deployment (and later on inference calls on multiple documents) are lenghty processes. When working with large models, the default `timeout` of 10s is not sufficient to avoid the operation being stopped even in cases where there are in fact to issues with the command.
> Add `timout='1m'` 

Use the `wait_for` arguments to allow lenghtier processes to run in the background and check the status later rather than allowing the task to idle.


### Queue Capacity

>Each allocation of a model deployment has a dedicated queue to buffer inference requests. The size of this queue is determined by the queue_capacity parameter in the start trained model deployment API. When the queue reaches its maximum capacity, new requests are declined until some of the queued requests are processed, creating available capacity once again. When multiple ingest pipelines reference the same deployment, the queue can fill up, resulting in rejected requests. Consider using dedicated deployments to prevent this situation.

>queue_capacity

>(Optional, integer) Controls how many inference requests are allowed in the queue at a time. Every machine learning node in the cluster where the model can be allocated has a queue of this size; when the number of requests exceeds the total value, new requests are rejected with a 429 error. Defaults to 1024. Max allowed value is 1000000.

https://www.elastic.co/guide/en/elasticsearch/reference/current/start-trained-model-deployment.html#start-trained-model-deployment-deployment-id-example

You can deploy multiple instances of the same model, each with a unique `deployment_id="my_elser_model_for_inference"` to then specify which instance you are sending a request to. This can help you better manage the queues of your models to avoid rejected requests. 

```
client.ml.start_trained_model_deployment(
    model_id=model_id, 
    deployment_id="elser_inference_1",
    queue_capacity="7000",
    }
client.ml.start_trained_model_deployment(
    model_id=model_id, 
    deployment_id="elser_inference_2",
    queue_capacity="5000",
    }
```
And choose which allocation of the model to run a particular inference on by using the unique `deployment_id` on the `model_id` field. 

```
client.ml.infer_trained_model(
    model_id="my_elser_model_for_movies_use_case",
    body={"docs": [{"text_field": "The movie was awesome!!"}]},
)
```


### Threads & Allocation

See section above describing setting these properties on your ML Node. You then have the option to define how many of the available threads & allocation each deployed model should leverage. 

From this notebook: https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/model-upgrades/upgrading-index-to-use-elser.ipynb 

## Bottleneck Opportunities Phase 3: Inference

Once the model is deployed, you can start making inference calls to it. This can be done via the Inference API:

```
response = client.ml.infer_trained_model(
    # Using the deployment_id here to specify which version of our model we want to send the inference to (as we plan to have multiple deployments of the same model_id in this project)
    model_id="elser_inference_1", 
    body={
        "docs": [
            {
                "text_field": "This movie was totally awesome and had loads of adventure"
            }
        ]
    }
)
print(response["inference_results"][0]["predicted_value"])
```

However, for most real use cases, there will be a lot of documents that need to be run through the inference API. For instance, with an embedding model such as ELSER, if you want to perform semantic search against an index, each of the documents in the index must first be embedded. 

To generate embeddings for each document, you can run an inference pipeline. In simple cases like in the notebook example, using 12 documents, this will run quite smoothly out of the box, with a minimally configured model deployment. 
However, with larger projects, we can see that the default values are no longer sufficient to keep up with the throughout. 

Here are some potential bottlenecks for the inference process and how to prevent them:

The first parameter to be mindful of is similar to the `wait_for` we've seen previously for the deployment status:
>`wait_for_completion = False`

This can be added to the `reindex` command that triggers the embedding pipeline. With a default timeout after just 10s, in the case of a pipeline that has to run inference calls on thousasnds of documents in a queue, this will most definetely end in an error without setting the wait argument to False. 




Monitor thread usage
`GET _nodes/hot_threads `

> 100.0% [cpu=3.5%, other=96.5%] (500ms out of 500ms) cpu usage by thread 'elasticsearch[instance-0000000010][ml_native_inference_comms][T#12]'
> ml.allocated_processors=16



In [26]:
from urllib.request import urlopen
from urllib.request import urlopen
import json
import time

client.ingest.put_pipeline(
    id="ingest-pipeline-lowercase",
    description="Ingest pipeline to change title to lowercase",
    processors=[{"lowercase": {"field": "title"}}],
)

client.indices.delete(index="movies", ignore_unavailable=True)
client.indices.create(
    index="movies",
    settings={
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 1,
            "default_pipeline": "ingest-pipeline-lowercase",
        }
    },
    mappings={
        "properties": {
            "plot": {
                "type": "text",
                "fields": {"keyword": {"type": "keyword", "ignore_above": 256}},
            },
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'})

In [27]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append(
        {
            "_index": "movies",
            "_source": doc,
        }
    )

# Use helpers.bulk to index
helpers.bulk(client, documents)

time.sleep(5)
print("Done indexing documents into `movies` index!")

Done indexing documents into `movies` index!


In [ ]:
models = client.ml.get_trained_models()
for model in models["trained_model_configs"]:
    print(model["model_id"])

In [ ]:
resp = client.ml.get_trained_models(model_id=".elser_model_1")
print(resp)

In [ ]:
resp = client.ml.get_trained_models_stats()
print(resp)

In [ ]:
resp = client.ml.put_trained_model_alias(
    model_id="distilbert-base-uncased-finetuned-sst-2-english",
    model_alias="sentiment",
)
print(resp)

In [18]:
model_id="elser_inference_1"
resp = client.ml.infer_trained_model(
    model_id=model_id, 
    docs=[{"text_field": "The movie was awesome!!"}],
)
print(resp["inference_results"][0]["predicted_value"])


{'awesome': 2.3845856, 'amazing': 2.0961068, 'movie': 2.0593815, '!': 1.1960324, 'incredible': 1.0804834, 'ok': 1.0448649, 'epic': 0.89339185, 'was': 0.80329055, 'gross': 0.76922053, 'award': 0.748162, 'the': 0.72624725, 'inspired': 0.715019, 'film': 0.682253, 'soundtrack': 0.67788297, 'scene': 0.66170484, 'exclusive': 0.6343134, 'theater': 0.62419486, 'humor': 0.4557575, 'hollywood': 0.4256505, 'oscar': 0.41349027, 'wonderful': 0.40632337, 'overall': 0.39366663, 'great': 0.38513696, 'sequel': 0.33310017, 'audience': 0.32106632, 'surprise': 0.26039606, 'stunt': 0.24162033, 'okay': 0.23190367, 'horror': 0.22539854, 'disney': 0.21497314, 'wow': 0.2060134, 'success': 0.19813244, 'movies': 0.17897457, 'kid': 0.15326343, 'actor': 0.13542877, 'performance': 0.12722221, 'original': 0.110997565, 'broadway': 0.10838837, 'bomb': 0.10409451, 'review': 0.096696295, 'fame': 0.086990975, 'director': 0.079721816, 'video': 0.0535536, 'concert': 0.048921354, 'commercial': 0.044184294, 'everyone': 0.035

In [ ]:
resp = client.ml.infer_trained_model(
    model_id=".elser_model_1",
    docs = [{"text_field": "iulia is the best engineer on the ml team"}],
)
print(resp)

In [ ]:
question = ''

In [22]:
result = client.search(
    index='elser-movies', 
    size=5,
    query={
        "text_expansion": {
            "ml.tokens": {
                "model_id":".elser_model_2",
                "model_text":"dramatic movie with fight"
            }
        }
    }
)

for element in result["hits"]["hits"]:
    print("{}: {}, score {}".format(element["_source"]["plot_embeddings"], element["_score"]))

In [25]:
client.tasks.get(task_id=task_id)

ObjectApiResponse({'completed': True, 'task': {'node': 'JqYuDbWsRueybLrxY3c9Cg', 'id': 40847263, 'type': 'transport', 'action': 'indices:data/write/reindex', 'status': {'total': 12, 'updated': 0, 'created': 12, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0}, 'description': 'reindex from [movies] to [elser-movies]', 'start_time_in_millis': 1712145479211, 'running_time_in_nanos': 617921707, 'cancellable': True, 'cancelled': False, 'headers': {}}, 'response': {'took': 616, 'timed_out': False, 'total': 12, 'updated': 0, 'created': 12, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled': '0s', 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until': '0s', 'throttled_until_millis': 0, 'failures': []}})

In [3]:
client.ingest.get_pipeline(id="sentiment1")

ObjectApiResponse({'sentiment1': {'processors': [{'inference': {'model_id': 'distilbert-base-uncased-finetuned-sst-2-english', 'target_field': 'sentiment', 'field_map': {'Sentence': 'text_field'}}}]}})

In [33]:
resp = client.render_search_template(
    id="my-search-template",
    params={"from": 20, "size": 10}
)
print(resp)

BadRequestError: BadRequestError(400, 'search_phase_execution_exception', 'runtime error')